In [1]:
import sys, os
sys.path.append('..')

# std
import numpy as np
import random as rn
import json
import time
from collections import defaultdict
from itertools import combinations

# datasets
import STRING
import MINT
import bioGRID
import HuRI

# my lib
import PPILinkPred as pred
import genData_helper as helper
import traversalHelper as tr
import helper as hr

# Generate Random PPI Samples from Datasets

In [2]:
# yeast dataset
# DataFrame standard: {nodeA, nodeB, type, score}
# randomly 50% of the dataset 10 times, save into json

import_funcs = [bioGRID.parse_bioGRID(root="../"), STRING.parse_STRING(root="../"), MINT.parse_MINT(root="../")]
names = ['bioGRID', 'STRING', 'MINT']

for n in range(len(names)):
    _, df = import_funcs[n]
    ppi = [list(arr) for arr in np.asarray(df[['nodeA', 'nodeB']])]
    sampledPPIs = [rn.sample(ppi, int(len(ppi)*0.5)) for i in range(10)]
    with open("./sampled_datasets/{}_sampledPPIs.json".format(names[n]), "w") as f:
        f.write(json.dumps(sampledPPIs))

In [2]:
# yeast dataset
# DataFrame standard: {nodeA, nodeB, type, score}
# randomly 50% of the dataset 10 times, save into json

import_funcs = [bioGRID.parse_bioGRID(root="../"), STRING.parse_STRING(root="../"), MINT.parse_MINT(root="../")]
names = ['bioGRID', 'STRING', 'MINT']

for randSz in range(95, 54, -5):
    for n in range(len(names)):
        _, df = import_funcs[n]
        ppi = [list(arr) for arr in np.asarray(df[['nodeA', 'nodeB']])]
        sampledPPIs = [rn.sample(ppi, int(len(ppi)*(randSz*0.01))) for i in range(10)]
        with open("./sampled_datasets/{}_sampledPPIs_{}Percent.json".format(names[n], randSz), "w") as f:
            f.write(json.dumps(sampledPPIs))

In [3]:
# yeast dataset
# sample non-PPIs of real-PPIs size

import_funcs = [bioGRID.parse_bioGRID(root="../"), STRING.parse_STRING(root="../"), MINT.parse_MINT(root="../")]
names = ['bioGRID', 'STRING', 'MINT']

for n in range(len(names)):
    _, df = import_funcs[n]
    ppi = [list(arr) for arr in np.asarray(df[['nodeA', 'nodeB']])]
    realPPIsz = len(ppi)
    dual_ppiStr = set(tr.Helper.list_to_pathStrs(tr.Helper.to_dual_binary_relation(ppi)))
    
    sampleNodes = tr.Helper.binary_relation_to_node(ppi)
    nodePairs = tr.Helper.list_to_pathStrs(list(combinations(sampleNodes, 2)))
    
    nonPPIs = [ppi for ppi in nodePairs if ppi not in dual_ppiStr]
    sampled_nonPPIs = [tr.Helper.pathStrs_to_list(rn.sample(nonPPIs, realPPIsz)) for i in range(10)]
    
    with open("./sampled_datasets/{}_sampled_nonPPIs.json".format(names[n]), "w") as f:
        f.write(json.dumps(sampled_nonPPIs))

# Run Link Prediction

In [ ]:
methods = ["commonNeighbor", "L3Normalizing", "CRA", "CH2_L3", "Sim", 'L3E1_f1', 'L3E1_f2', 'random']
ds_names = ['bioGRID', 'STRING', 'MINT']

for ds_name in ds_names:
    # read dataset
    samplePPIs = []
    with open("./sampled_datasets/{}_sampledPPIs.json".format(ds_name), "r") as f:
        samplePPIs = json.loads(f.read())

    # do link prediction & save results
    for method in methods:
        for i in range(len(samplePPIs)):
            saveFilename = "{}_{}_sample_{}".format(method, ds_name, i)
            startTime = time.time()
            
            # jupyter notebook cannot display multi core logging, do it only in terminal
            predPPI, predScore = pred.multiCore_PPILinkPred(samplePPIs[i]
                                                            , method, coreNo=14, logging=False)
            helper.write_runTime(saveFilename, time.time()-startTime)
            helper.write_resultData(predPPI, predScore, saveFilename)

In [ ]:
methods = ["commonNeighbor", "L3Normalizing", "CRA", "Sim", "L3E1_f1", 'L3E1_f2', 'random']
# skip CH2 and L3E1_f2 first because waste time, may use HPC
ds_names = ['bioGRID', 'STRING', 'MINT']

for randSz in range(60, 100, 10):
    for ds_name in ds_names:
        # read dataset
        samplePPIs = []
        with open("./sampled_datasets/{}_sampledPPIs_{}Percent.json".format(ds_name, randSz), "r") as f:
            samplePPIs = json.loads(f.read())

        # do link prediction & save results
        for method in methods:
            print(randSz, ds_name, method)
            for i in range(len(samplePPIs)):
                saveFilename = "{}_{}_sample_{}_randSz{}Percent".format(method, ds_name, i, randSz)
                startTime = time.time()

                # jupyter notebook cannot display multi core logging, do it only in terminal
                predPPI, predScore = pred.multiCore_PPILinkPred(samplePPIs[i]
                                                                , method, coreNo=14, logging=False)
                helper.write_runTime(saveFilename, time.time()-startTime)
                helper.write_resultData(predPPI, predScore, saveFilename)

# Data Cleaning for Analysis & Processing

In [23]:
# extract only top n edges, n = size of sampled dataset
methods = ["commonNeighbor", "L3Normalizing", "CRA", "CH2_L3", "Sim", 'random']+[j+"_"+i for j in ['L3E1', 'L3E4'] for i in ['f1', 'f2']]
ds_names = ['bioGRID', 'STRING', 'MINT']

for ds_name in ds_names:
    samplePPIs = []
    with open("./sampled_datasets/{}_sampledPPIs.json".format(ds_name), "r") as f:
        samplePPIs = json.loads(f.read())
    sampleSize = len(samplePPIs[0])
    
    # loop each method, each trial, extract the number into one json
    for method in methods:
        fullPPI, fullScore = [], []
        for trial in range(10):
            with open("E:/research/ppiLPred_BMC/notebook/linkPred_out/{}_{}_sample_{}_PPI.json".format(method, ds_name, trial), "r") as f:
                fullPPI.append(json.loads(f.read())[0:sampleSize])
            with open("E:/research/ppiLPred_BMC/notebook/linkPred_out/{}_{}_sample_{}_score.json".format(method, ds_name, trial), "r") as f:
                fullScore.append(json.loads(f.read())[0:sampleSize])
        with open("./linkPred_out_reduced/{}_{}_topPPI.json".format(method, ds_name), "w") as f:
            f.write(json.dumps(fullPPI))
        with open("./linkPred_out_reduced/{}_{}_topScore.json".format(method, ds_name), "w") as f:
            f.write(json.dumps(fullScore))

In [2]:
allPaths = ["I:/research/ppiLPred_BMC/notebook"
            , "E:/research/ppiLPred_BMC/notebook"
            , "D:/research offline repo/ppiLPred_BMC/notebook"
            , "G:/research/ppiLPred_BMC/notebook"]
coreNo, trialNum = 24, 10
def verify(method, ds, randSz):
    # check HPC or not
    isHPC = None
    for path in allPaths:
        # check if file exists in linkPred_out_reduced
        if os.path.exists("./linkPred_out_reduced/{}_{}_randSz{}_topPPI.json".format(
            method, ds, randSz)): return 0, None, None
        if os.path.exists("{}/linkPred_out/{}_{}_sample_9_randSz{}Percent_c0_PPI.json".format(
            path, method, ds, randSz)):
            isHPC = True
            break
        elif os.path.exists("{}/linkPred_out/{}_{}_sample_9_randSz{}Percent_PPI.json".format(
            path, method, ds, randSz)):
            isHPC = False
            break
    if isHPC is None: return 2, None, None
    # iterate the abs path to all related files
    filenames = []
    if isHPC:
        for trial in range(trialNum):
            for core in range(coreNo):
                for path in allPaths:
                    filename = "{}/linkPred_out/{}_{}_sample_{}_randSz{}Percent_c{}_PPI.json".format(
                        path, method, ds, trial, randSz, core)
                    if os.path.exists(filename): filenames.append(filename.split("_PPI.json")[0])
    else:
        for trial in range(trialNum):
            for path in allPaths:
                filename = "{}/linkPred_out/{}_{}_sample_{}_randSz{}Percent_PPI.json".format(
                    path, method, ds, trial, randSz)
                if os.path.exists(filename): filenames.append(filename.split("_PPI.json")[0])
    # return available, list of files, also isHPC
    return 1, filenames, isHPC

In [11]:
# trim data that isn't trimmed yet
methods = ["commonNeighbor", "L3Normalizing", "CRA", "CH2_L3", "Sim", 'random', "L3E1_f1", "L3E1_f2"]
dss = ['bioGRID', 'STRING', 'MINT']
coreNo, trialNum = 24, 10

for randSz in range(60, 100, 10):
    for ds in dss:
        samplePPIs = []
        with open("./sampled_datasets/{}_sampledPPIs_{}Percent.json".format(ds, randSz), "r") as f:
            samplePPIs = json.loads(f.read())
        sampleSize = len(samplePPIs[0])
    
        for method in methods:
            available, filenames, isHPC = verify(method, ds, randSz)
            print(randSz, ds, method, isHPC, available)
            if available != 1: continue

            if isHPC:
                
                for trial in range(trialNum):
                    topPPIs, topScores = [], []
                    for core in range(coreNo):
                        with open(filenames[trial*coreNo+core]+"_PPI.json", "r") as f: topPPIs += json.loads(f.read())
                        with open(filenames[trial*coreNo+core]+"_score.json", "r") as f: topScores += json.loads(f.read())
                        topPPIs, topScores = hr.sort_key_val(topPPIs, topScores)
                        topPPIs, topScores = topPPIs[:sampleSize], topScores[:sampleSize]
                    with open("./linkPred_out_combined/{}_{}_sample_{}_randSz{}_topPPI.json".format(method, ds, trial, randSz), "w") as f:
                        f.write(json.dumps(topPPIs))
                    with open("./linkPred_out_combined/{}_{}_sample_{}_randSz{}_topScore.json".format(method, ds, trial, randSz), "w") as f:
                        f.write(json.dumps(topScores))
                        
                fullPPIs, fullScores = [], []
                for trial in range(10):
                    with open("./linkPred_out_combined/{}_{}_sample_{}_randSz{}_topPPI.json".format(method, ds, trial, randSz), "r") as f:
                        fullPPIs.append(json.loads(f.read()))
                    with open("./linkPred_out_combined/{}_{}_sample_{}_randSz{}_topScore.json".format(method, ds, trial, randSz), "r") as f:
                        fullScores.append(json.loads(f.read()))
                with open("./linkPred_out_reduced/{}_{}_randSz{}_topPPI.json".format(method, ds, randSz), "w") as f:
                    f.write(json.dumps(fullPPIs))
                with open("./linkPred_out_reduced/{}_{}_randSz{}_topScore.json".format(method, ds, randSz), "w") as f:
                    f.write(json.dumps(fullScores))
                    
            else:
                
                fullPPIs, fullScores = [], []
                for filename in filenames:
                    with open(filename+"_PPI.json", "r") as f: fullPPIs.append(json.loads(f.read())[0:sampleSize])
                    with open(filename+"_score.json", "r") as f: fullScores.append(json.loads(f.read())[0:sampleSize])
                with open("./linkPred_out_reduced/{}_{}_randSz{}_topPPI.json".format(method, ds, randSz), "w") as f:
                    f.write(json.dumps(fullPPIs))
                with open("./linkPred_out_reduced/{}_{}_randSz{}_topScore.json".format(method, ds, randSz), "w") as f:
                    f.write(json.dumps(fullScores))

60 bioGRID commonNeighbor False 1
60 bioGRID L3Normalizing False 1
60 bioGRID CRA False 1
60 bioGRID CH2_L3 True 1
60 bioGRID Sim False 1
60 bioGRID random False 1
60 bioGRID L3E1_f1 False 1
60 bioGRID L3E1_f2 True 1
60 STRING commonNeighbor False 1
60 STRING L3Normalizing False 1
60 STRING CRA False 1
60 STRING CH2_L3 True 1
60 STRING Sim False 1
60 STRING random False 1
60 STRING L3E1_f1 False 1
60 STRING L3E1_f2 True 1
60 MINT commonNeighbor False 1
60 MINT L3Normalizing False 1
60 MINT CRA False 1
60 MINT CH2_L3 True 1
60 MINT Sim False 1
60 MINT random False 1
60 MINT L3E1_f1 False 1
60 MINT L3E1_f2 True 1
70 bioGRID commonNeighbor False 1
70 bioGRID L3Normalizing False 1
70 bioGRID CRA False 1
70 bioGRID CH2_L3 True 1
70 bioGRID Sim False 1
70 bioGRID random False 1
70 bioGRID L3E1_f1 False 1
70 bioGRID L3E1_f2 True 1
70 STRING commonNeighbor False 1
70 STRING L3Normalizing False 1
70 STRING CRA False 1
70 STRING CH2_L3 True 1
70 STRING Sim False 1
70 STRING random False 1
70 STR

In [35]:
# above randSz reduced wrong, reduce even more
methods = ["commonNeighbor", "L3Normalizing", "CRA", "CH2_L3", "Sim", 'random', "L3E1_f1", "L3E1_f2"]
dss = ['bioGRID', 'STRING', 'MINT']

for randSz in range(60, 100, 10):
    for ds in dss:
        samplePPIs = []
        with open("./sampled_datasets/{}_sampledPPIs_{}Percent.json".format(ds, randSz), "r") as f:
            samplePPIs = json.loads(f.read())
        sampleSize = len(samplePPIs[0])
    
        for method in methods:
            reducedPPIs, reducedScores = [], []
            with open("./linkPred_out_reduced/{}_{}_randSz{}_topPPI.json".format(method, ds, randSz), "r") as f:
                reducedPPIs = json.loads(f.read())
            with open("./linkPred_out_reduced/{}_{}_randSz{}_topScore.json".format(method, ds, randSz), "r") as f:
                reducedScores = json.loads(f.read())
                
            for i in range(len(reducedPPIs)):
                reducedPPIs[i] = reducedPPIs[i][:int(sampleSize/(randSz/100)*np.around(1-randSz/100, 2))]
                reducedScores[i] = reducedScores[i][:int(sampleSize/(randSz/100)*np.around(1-randSz/100, 2))]
                
            with open("./linkPred_out_reduced/{}_{}_randSz{}_topPPI.json".format(method, ds, randSz), "w") as f:
                f.write(json.dumps(reducedPPIs))
            with open("./linkPred_out_reduced/{}_{}_randSz{}_topScore.json".format(method, ds, randSz), "w") as f:
                f.write(json.dumps(reducedScores))

# Generate GOSemSim

run **GOSemSim_compute.R** of the same directory, it scans ./linkPred_out and output GOSemSim in the same format of **xxx_topScore.json**

# Generate precision recall

In [5]:
# for each dataset & predictor, get precision recall and save in one file for each trial
methods = ["commonNeighbor", "L3Normalizing", "CRA", "CH2_L3", "Sim", "L3E1_f1", "L3E1_f2", "random"]
ds_names = ['bioGRID', 'STRING', 'MINT']

completePPIs_map = [
    [list(ppi) for ppi in np.asarray([*bioGRID.parse_bioGRID(root='../')][1][['nodeA', 'nodeB']])]
    , [list(ppi) for ppi in np.asarray([*STRING.parse_STRING(root='../')][1][['nodeA', 'nodeB']])]
    , [list(ppi) for ppi in np.asarray([*MINT.parse_MINT(root='../')][1][['nodeA', 'nodeB']])]
]
completePPIs = dict(zip(ds_names, completePPIs_map))


for ds_name in ds_names:
    samplePPIs = []
    with open("./sampled_datasets/{}_sampledPPIs.json".format(ds_name), "r") as f:
        samplePPIs = json.loads(f.read())

    for method in methods:
        fullPPIs = []
        with open("./linkPred_out_reduced/{}_{}_topPPI.json".format(method, ds_name), "r") as f:
            fullPPIs = json.loads(f.read())
            
        # len(fullPPIs) = len(samplePPIs) = 10, because 10 trials
        precRecMap = pred.precRecMap_multiCore(
            ["{}_{}_topPPI_{}".format(method, ds_name, i) for i in range(len(fullPPIs))]
          , fullPPIs, samplePPIs, [completePPIs[ds_name] for i in range(len(fullPPIs))]
          , coreNo=10)
        
        for key in precRecMap:
            with open("./precision_recall_out/{}.json".format(key), 'w') as f:
                f.write(json.dumps(precRecMap[key]))

In [36]:
# for each dataset & predictor, get precision recall and save in one file for each trial
methods = ["commonNeighbor", "L3Normalizing", "CRA", "CH2_L3", "Sim", "random", "L3E1_f1", "L3E1_f2"]
ds_names = ['bioGRID', 'STRING', 'MINT']

completePPIs_map = [
    [list(ppi) for ppi in np.asarray([*bioGRID.parse_bioGRID(root='../')][1][['nodeA', 'nodeB']])]
    , [list(ppi) for ppi in np.asarray([*STRING.parse_STRING(root='../')][1][['nodeA', 'nodeB']])]
    , [list(ppi) for ppi in np.asarray([*MINT.parse_MINT(root='../')][1][['nodeA', 'nodeB']])]
]
completePPIs = dict(zip(ds_names, completePPIs_map))

for randSz in range(60, 100, 10):
    for ds_name in ds_names:
        samplePPIs = []
        with open("./sampled_datasets/{}_sampledPPIs_{}Percent.json".format(ds_name, randSz), "r") as f:
            samplePPIs = json.loads(f.read())

        for method in methods:
            fullPPIs = [] 
            with open("./linkPred_out_reduced/{}_{}_randSz{}_topPPI.json".format(method, ds_name, randSz), "r") as f:
                fullPPIs = json.loads(f.read())

            # len(fullPPIs) = len(samplePPIs) = 10, because 10 trials
            precRecMap = pred.precRecMap_multiCore(
                ["{}_{}_randSz{}_topPPI_{}".format(method, ds_name, randSz, i) for i in range(len(fullPPIs))]
              , fullPPIs, samplePPIs, [completePPIs[ds_name] for i in range(len(fullPPIs))]
              , coreNo=10)

            for key in precRecMap:
                with open("./precision_recall_out/{}.json".format(key), 'w') as f:
                    f.write(json.dumps(precRecMap[key]))